In [4]:
import geopandas as gpd
import os

# Base directory path
base_dir = r"C:\Users\natda\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths using os.path.join
junctions_path = os.path.join(base_dir, "junctions.shp")
signals_path = os.path.join(base_dir, "osm_signals.gdb")

# Load shapefiles
junctions = gpd.read_file(junctions_path)
signals = gpd.read_file(signals_path)

# Reproject signals to match junctions CRS (EPSG:6491)
signals = signals.to_crs(junctions.crs)

# Create the 'control' column in the junctions shapefile
junctions['control'] = ""

# Add 'ID' column to signals and assign sequential values
signals['ID'] = range(1, len(signals) + 1)

# List to track unassociated signals
unassociated_signals = set(signals['ID'].tolist())

In [5]:
# Function to calculate the distance between a signal and all junctions
def find_nearest_junction(signal, junctions_gdf, threshold=25):
    # Calculate distances to all junctions
    distances = junctions_gdf.geometry.distance(signal.geometry)
    within_threshold = distances[distances <= threshold]
    
    if within_threshold.empty:
        return None  # No junctions within 14 m
    
    # Find the closest junction
    nearest_junction_idx = within_threshold.idxmin()
    return junctions_gdf.loc[nearest_junction_idx, 'geometry'], nearest_junction_idx

# Iterate through each signal and find the closest junction
for _, signal in signals.iterrows():
    nearest = find_nearest_junction(signal, junctions)
    
    if nearest:
        _, nearest_junction_idx = nearest
        
        # Mark the nearest junction as "signal" in the control column
        junctions.at[nearest_junction_idx, 'control'] = "signal"
        
        # Remove this signal ID from the unassociated list
        unassociated_signals.discard(signal['ID'])

# Save the updated junctions shapefile
junctions.to_file(junctions_path, driver='ESRI Shapefile')

# Output the list of unassociated signal IDs
print(f"There are {len(unassociated_signals)} unassociated signals.")
print("Unassociated Signals:", list(unassociated_signals))

There are 414 unassociated signals.
Unassociated Signals: [1, 2, 3, 4, 5, 6, 42, 188, 202, 205, 223, 229, 248, 266, 283, 302, 316, 325, 339, 342, 370, 380, 383, 397, 406, 407, 416, 419, 420, 431, 433, 439, 442, 445, 446, 447, 451, 453, 454, 459, 476, 480, 481, 491, 500, 506, 515, 517, 526, 527, 534, 537, 538, 550, 552, 555, 557, 560, 566, 567, 568, 569, 572, 573, 574, 575, 576, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 596, 598, 600, 602, 606, 608, 610, 611, 612, 613, 614, 615, 616, 617, 622, 625, 627, 630, 632, 638, 645, 658, 659, 660, 670, 672, 674, 675, 676, 686, 687, 690, 691, 692, 707, 709, 710, 715, 721, 724, 726, 730, 731, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 755, 757, 758, 759, 760, 761, 765, 766, 767, 768, 769, 770, 771, 774, 777, 778, 789, 807, 809, 811, 814, 817, 822, 823, 824, 825, 833, 834, 835, 836, 841, 848, 852, 859, 861, 862, 868, 869, 870, 888, 957, 958, 969, 989, 990, 991, 996, 1003, 1006, 1009, 1029, 1030, 1045, 1058, 1059, 1078, 1079,

In [6]:
junctions['control'].value_counts()

control
          11289
signal     1058
Name: count, dtype: int64